Methodology
4.1 Data Science Workflow/Framework
4.1.1. Data Collection
4.1.2 Data Preprocessing
4.1.3 Data Exploration
4.1.4 Data Modelling
4.1.5 Model Evaluation

In [1]:
import os
import json
from flatten_json import flatten
from PIL import Image, ImageDraw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
expert_path = 'tufts-dental-database/Expert/expert.json'
expert_df = pd.read_json(expert_path)
expert_df.head()

,Label,Description,External ID
0,"{'objects': [{'title': 'None', 'value': 'none'...",Within normal limits,53.JPG
1,"{'objects': [{'title': 'None', 'value': 'none'...",Within normal limits,823.JPG
2,"{'objects': [{'title': 'None', 'value': 'none'...",Within normal limits,85.JPG
3,"{'objects': [{'title': 'None', 'value': 'none'...",Within normal limits,855.JPG
4,"{'objects': [{'title': 'None', 'value': 'none'...",Within normal limits,117.JPG


In [3]:
expert_df.Label

0      {'objects': [{'title': 'None', 'value': 'none'...
1      {'objects': [{'title': 'None', 'value': 'none'...
2      {'objects': [{'title': 'None', 'value': 'none'...
3      {'objects': [{'title': 'None', 'value': 'none'...
4      {'objects': [{'title': 'None', 'value': 'none'...
                             ...                        
995    {'objects': [{'title': 'Periapical', 'value': ...
996    {'objects': [{'title': 'Non-Odontogenic', 'val...
997    {'objects': [{'title': 'None', 'value': 'none'...
998    {'objects': [{'title': 'None', 'value': 'none'...
999    {'objects': [{'title': 'Periapical', 'value': ...
Name: Label, Length: 1000, dtype: object

In [4]:
a = pd.json_normalize(expert_df.Label)
a = pd.json_normalize(a.objects)
a = pd.json_normalize(a[0])
# a = pd.json_normalize(a['classifications'])
a

,title,value,classifications,polygons
0,None,none,none,none
1,None,none,none,none
2,None,none,none,none
3,None,none,none,none
4,None,none,none,none
...,...,...,...,...
995,Periapical,periapical,"[{'featureId': 'ck4ejta67049t0xg2xmuudqqw', 's...","[[[493, 562], [492, 563], [491, 563], [490, 56..."
996,Non-Odontogenic,non-odontogenic,"[{'featureId': 'ck4ejuf3204ak0xg2iu8qjc9s', 's...","[[[1473, 701], [1473, 706], [1472, 707], [1472..."
997,None,none,none,none
998,None,none,none,none


904.jpg
inflammation on 18, 21, 20

In [5]:
radiograph = 'tufts-dental-database/Radiographs'
segmentation = 'tufts-dental-database/Segmentation'
expert = 'tufts-dental-database/Expert'
student = 'tufts-dental-database/Student'

radiograph_images = os.listdir(radiograph)
radiograph_expert_masks = os.listdir(expert+'/mask')

In [6]:
def draw_polygon_mask(xy_per_polygon, img_path):
    xray = Image.open(img_path)


    for xy in xy_per_polygon:
        abnormality_mask = xray.copy()
        draw = ImageDraw.Draw(abnormality_mask)
        draw.polygon(xy, fill ="#eeeeff", outline ="blue")

        xray = Image.blend(xray, abnormality_mask, 0.5)
    return xray

# x = [100, 200, 100]
# y = [150, 450, 700]
# xy = list(zip(x,y))
# draw_polygon_mask(xy, 'bdsm meme.jpg')

In [18]:
sample = expert_df[expert_df['External ID'] == '1002.JPG']
sample = pd.json_normalize(sample['Label'])
sample = sample['objects'][0]
sample = pd.json_normalize(sample)
sample = sample['polygons'][0]
mask_1 = sample[0]
mask_1 = list(map(tuple, mask_1))
# mask_2 = sample[1]
# mask_2 = list(map(tuple, mask_2))

In [19]:
xray_sample = radiograph +'/1002.jpg'
xray_sample_mask = expert + '/mask/1002.jpg'

In [20]:
# xray
a = Image.open(xray_sample)
a.show()
# mask
b = Image.open(xray_sample_mask)
b.show()
# overlayed mask
a.paste(b, mask=b)
a.show()
# overlayed polygon mask
c = draw_polygon_mask([mask_1], xray_sample)
c.show()